Для решения анафоры разобъем файлы на директории, содержащие файлы по k предложений

In [37]:
import os
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
from concurrent.futures import ThreadPoolExecutor, as_completed

In [38]:
def split_text_into_sentences(text):
    """
    Разбивает текст на предложения.
    :param text: Исходный текст.
    :return: Список предложений.
    """
    return sent_tokenize(text)

def group_sentences(sentences, group_size=10):
    """
    Группирует предложения по заданному размеру.
    :param sentences: Список предложений.
    :param group_size: Размер группы (по умолчанию 10).
    :return: Список групп предложений.
    """
    return [sentences[i:i + group_size] for i in range(0, len(sentences), group_size)]

def process_file(file, input_directory, output_directory, group_size):
    """
    Обрабатывает один файл: разбивает его на части и сохраняет в выходную директорию.
    :param file: Путь к входному файлу.
    :param input_directory: Директория с входными файлами.
    :param output_directory: Директория для выходных файлов.
    :param group_size: Количество предложений в каждой части.
    """
    try:
        # Читаем текст из входного файла
        with open(file, "r", encoding="utf-8") as f:
            text = f.read()

        # Разбиваем текст на предложения
        sentences = split_text_into_sentences(text)

        # Группируем предложения по group_size
        groups = group_sentences(sentences, group_size)

        # Вычисляем относительный путь файла относительно input_directory
        relative_path = file.relative_to(input_directory)

        # Создаем путь к выходной директории, сохраняя структуру поддиректорий
        output_subdir = output_directory / relative_path.parent / file.stem
        os.makedirs(output_subdir, exist_ok=True)

        # Сохраняем каждую группу в отдельный файл
        for i, group in enumerate(groups):
            output_path = output_subdir / f"part_{i + 1}.txt"
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(" ".join(group))

    except Exception as e:
        print(f"Error processing file {file}: {e}")

def split_files(input_directory, output_directory, group_size=10, max_workers=8):
    """
    Разбивает файлы на части по group_size предложений и сохраняет в директории.
    Использует многопоточность для ускорения.
    :param input_directory: Директория с входными файлами.
    :param output_directory: Директория для выходных файлов.
    :param group_size: Количество предложений в каждой части.
    :param max_workers: Максимальное количество потоков.
    """
    # Получаем список всех txt файлов рекурсивно (включая поддиректории)
    input_files = list(input_directory.rglob("*.txt"))

    # Используем ThreadPoolExecutor для многопоточной обработки
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for file in input_files:
            futures.append(executor.submit(
                process_file, file, input_directory, output_directory, group_size
            ))

        # Отслеживаем прогресс выполнения задач
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing files"):
            try:
                future.result()  # Проверяем результат выполнения задачи
            except Exception as e:
                print(f"Error during processing: {e}")

In [33]:
import os
from pathlib import Path

In [ ]:
# input_directory = Path("/home/simon/Desktop/concept-tree/concept-tree/nn_graph_makers/test_files")
# output_directory = Path("/home/simon/Desktop/concept-tree/concept-tree/nn_graph_makers/test_files_output")

input_directory = Path("/home/kdemyokhin_1/concept-tree-course-work/articles_parsed/arxiv-txt-cs")
output_directory = Path("/home/kdemyokhin_1/concept-tree-course-work/articles_parsed_splitted/arxiv-txt-cs")

In [42]:
split_files(input_directory, output_directory, group_size=8, max_workers=16)

Processing files: 100%|██████████| 100/100 [00:00<00:00, 1923.79it/s]
